# Numba Testing

In this notebook I will test the effect of using numba for optimization of functions in VolPy


### Step 0:
Importing all necessary libraries and specifying the inputs

In [1]:
import timeit as ti

In [2]:
import os
import volpy as vp
import numpy as np
import pyvista as pv

pc_path = os.path.relpath('../SampleData/PointCloud.csv')
pc_n = vp.nbdata.scatter(np.array([[0,0,0],[100,100,100]]), 1000)
pc_s = vp.scatter(np.array([[0,0,0],[100,100,100]]), 1000)

In [3]:
stp = """\
import os
import volpy as vp
import numpy as np
import pyvista as pv

pc_path = os.path.relpath('../SampleData/PointCloud.csv')
pc_n = vp.nbdata.scatter(np.array([[0,0,0],[100,100,100]]), 1000)
pc_s = vp.scatter(np.array([[0,0,0],[100,100,100]]), 1000)
"""

### Step 1:
Running the sampling algorithm and asking the function to output the ray origins as well

In [4]:
t1 = ti.timeit('l = pc_n.regularize([1, 1, 1], closed=True)', setup=stp, number=100)
t0 = ti.timeit('l = pc_s.regularize([1, 1, 1], closed=True)', setup=stp, number=100)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Invalid use of Function(<class 'tuple'>) with argument(s) of type(s): (list(array(float64, 2d, C)))
 * parameterized
In definition 0:
    All templates rejected with literals.
In definition 1:
    All templates rejected without literals.
This error is usually caused by passing an argument of a type that is unsupported by the named function.
[1] During: resolving callee type: Function(<class 'tuple'>)
[2] During: typing of call at /Users/shervinazadi/Documents/Git/VolPy/src/volpy/nbdata.py (491)


File "../../src/volpy/nbdata.py", line 491:
def np_voxelate_closed(cloud, unit):
    <source elided>
    vox_ind = [np.rint(point_scaled + unit * axes[n] * s * 0.001) for n in range(axes.shape[0]) for s in [-1, 1]]
    vox_ind_tup = tuple(vox_ind)
    ^


In [5]:
print("standard time:", t0)
print("numba time:", t1)

NameError: name 't0' is not defined

In [5]:
ln = pc_n.regularize([1, 1, 1], closed=True)
ls = pc_s.regularize([1, 1, 1])
print(np.sum(ln))
print(np.sum(ls))

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Invalid use of Function(<built-in function array>) with argument(s) of type(s): (list(array(float64, 2d, C)))
 * parameterized
In definition 0:
    TypingError: array(float64, 2d, C) not allowed in a homogeneous sequence
    raised from /Users/shervinazadi/opt/miniconda3/envs/py3-hou/lib/python3.7/site-packages/numba/typing/npydecl.py:472
In definition 1:
    TypingError: array(float64, 2d, C) not allowed in a homogeneous sequence
    raised from /Users/shervinazadi/opt/miniconda3/envs/py3-hou/lib/python3.7/site-packages/numba/typing/npydecl.py:472
This error is usually caused by passing an argument of a type that is unsupported by the named function.
[1] During: resolving callee type: Function(<built-in function array>)
[2] During: typing of call at /Users/shervinazadi/Documents/Git/VolPy/src/volpy/nbdata.py (492)


File "../../src/volpy/nbdata.py", line 492:
def np_voxelate_closed(cloud, unit):
    <source elided

In [ ]:
### Step 2:
Voxelating the point cloud to construct a lattice out of it

In [7]:
lattice = sample_cloud.regularize(unit, closed=True)
print(type(lattice))
print(lattice.unit)
print(lattice.bounds)

<class 'volpy.data.lattice'>
[0.01 0.01 0.01]
[[-0.09  0.03 -0.06]
 [ 0.06  0.18  0.06]]


### Step 3:
Plotting

In [9]:

# initiating the plotter
p = pv.PlotterITK() # ITK plotter for interactivity within the python notebook (itkwidgets library is required)

# fast visualization of the point cloud
sample_cloud.fast_notebook_vis(p)

# fast visualization of the lattice
lattice.fast_notebook_vis(p)

# adding the base mesh: light blue
mesh = pv.read(mesh_path)
p.add_mesh(mesh, color='#abd8ff')

# adding the ray origins: dark blue
p.add_points(pv.PolyData(ray_origins), color='#004887')

# plotting
p.show()


Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…